## Design and Goal
There are four objects (bed, bench, chair, table). Participants viewed each object 20 times per run.
- Runs 1 & 2 -- reserved to conduct searchlight
- Runs 3 & 4 -- pretest phase 
- Four training runs involving practice drawing two of the trained objects. 
- Runs 5 & 6 -- posttest phase

Our goal is to understand the change in the similarity between representations of trained (vs control) objects in several anatomically-defined ROIs before and after training.

## Approach
#### Unanchored Pre-Post Representational Differentiation
Define 'representation' as cope maps generated upon fitting GLM to each object for each run. Start by building an object x voxel matrix (4xK) for each run and vertically concatenate the two runs in each phase. Using this, compute a correlation matrix. 

Consider M = off-diagonal 4x4 block [:4,4:8] of this matrix; ensure the rows are sorted such that the first two are the Trained, and the last two are the Control objects. Now take the the top-left 2x2 matrix within M and let's call it M-trained. The bottom-right 2x2 = M-control. The diagonal elements of M-trained (A,D depicted below) reflect the representational similarity for the *same* object between runs. The off diagonal elements of M-trained (B,C) reflect the similarity between different objects across runs. Mean of (B,C) - Mean(A,D) = Representational distance between objects in this phase.  
|__ A __|__ B __|  
|__ C __|__ D __|  
Do the above for the pretest, then for the posttest; the difference between the resulting values is an *unanchored* measure of how differentiated the the object representations have become between the pre and post-training phases:

*Unanchored Differentiation = (Post-Phase Representational Distance) - (Pre-Phase Representational Distance)*

By computing this measure of prepost differentiation again for the control objects and taking the difference between Trained Prepost Differentiation and Control Prepost Differentiation, we attempt to identify the amount of prepost differentiation that occurred specifically as a result of training. 

#### Anchored Pre-Post Representational Differentiation
Another approach to calculating prepost differentiation is to take the mean of the representational distances between different-phase trained object representations (eg the distance between the pre-phase representation of object 1 and the post-phase representation of object 2, and the distance between the pre-phase representation of object 2 and the post-phase representation of object 1) and then subtracting from this mean the representational distance between pre-phase trained object representations (the distance between the pre-phase representations of object 1 and object 2).

In terms of matrix computations, the key difference here from the unanchored measure is beginning with three matrices, and for the first two vertically concatenating corresponding runs of distinct phases (runs 3 and 5, and runs 4 and 6) instead of runs of the same phase. The third matrix is produced the same way the pre-phase object&run-by-voxel matrix is produced for measuring unanchored differentiation. Once representational distance is computed from each of these matrices, anchored prepost representational differentiation is found from the formula:  

*Anchored Differentation = mean((Runs 3&5 Representational Distance), (Runs 4&6 Representational Distance)) - (Pre-Phase Representational Distance)*

As for unanchored differentiation, computing this measure of prepost differentiation again for the control objects and taking the difference between Trained Prepost Differentiation and Control Prepost Differentiation, we attempt to identify the amount of prepost differentiation that occurred specifically as a result of training. 

# Setup

In [3]:
import numpy as np
from numpy import shape
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')

In [6]:
## add helpers to python path
import sys
if 'D:\\neurosketch-master\\python' not in sys.path:
    sys.path.append('D:\\neurosketch-master\\python')

## root paths
curr_dir = os.getcwd()
data_dir = 'D:\\data'
proj_dir = 'D:\\neurosketch-master'
results_dir = 'D:\\neurosketch-master\\csv'
nb_name = '1_recog_prepost_RSA.ipynb'

## module definitions
import analysis_helpers as helpers
ROIs = ['V1','V2','LOC','IT','fusiform','parahippo','PRC','ento','hipp','mOFC',
        'Frontal','Parietal','supraMarginal','Insula','postCentral','preCentral']
ROIs_formatted = ["V1", "V2", "LOC", "IT", "fusiform", "para\nhippo",  "PRC",  "ento", "hipp", 'mOFC',
                 'Frontal', 'Parietal', 'supra\nmarginal', 'Insula', 'post\nCentral', 'preCentral']
roi_dir = os.path.join(data_dir, 'copes\\roi')
cope_dir = os.path.join(data_dir, 'copes\\recog\\objectGLM')
sub_dirs = sorted(os.listdir(roi_dir))
helpers.roi_dir, helpers.cope_dir = roi_dir, cope_dir

# Similarity analysis

Produce anchored and unanchored measures of representational differentiation for each subject and each ROI and each condition, outputting a subject-by-roi arrays for each configuration of condition and differentiation measurement approach.

In [7]:
Tradiffpre, Condiffpre, Tradiffpost, Condiffpost = [], [], [], []
AnchoredTrainedDiff, AnchoredControlDiff = [], []

for roi in ROIs:
    _Tradiffpre, _Condiffpre, _Tradiffpost, _Condiffpost = [], [], [], []
    _AnchoredTrained, _AnchoredControl = [], []
        
    for s in sub_dirs:
        trawit_mean_pre, conwit_mean_pre, trabtw_mean_pre, conbtw_mean_pre = \
        helpers.compare_btw_wit_cond_similarity_across_runs(s,'pre',roi)
        trawit_mean_post, conwit_mean_post, trabtw_mean_post, conbtw_mean_post = \
        helpers.compare_btw_wit_cond_similarity_across_runs(s,'post',roi)
        
        trawit_mean_on, conwit_mean_on, trabtw_mean_on, conbtw_mean_on = \
        helpers.compare_btw_wit_cond_similarity_across_runs(s,'35',roi)
        trawit_mean_off, conwit_mean_off, trabtw_mean_off, conbtw_mean_off = \
        helpers.compare_btw_wit_cond_similarity_across_runs(s,'46',roi)
        
        _Tradiffpre.append(trabtw_mean_pre - trawit_mean_pre)
        _Condiffpre.append(conbtw_mean_pre - conwit_mean_pre)
        
        _Tradiffpost.append(trabtw_mean_post - trawit_mean_post)
        _Condiffpost.append(conbtw_mean_post - conwit_mean_post)
    
        _AnchoredTrained.append(
            np.mean([(trabtw_mean_on - trawit_mean_on), (trabtw_mean_off - trawit_mean_off)]) -
            (trabtw_mean_pre - trawit_mean_pre))
        _AnchoredControl.append(
            np.mean([(conbtw_mean_on - conwit_mean_on), (conbtw_mean_off - conwit_mean_off)]) -
            (conbtw_mean_pre - conwit_mean_pre))
        
    _Tradiffpre,_Condiffpre, _Tradiffpost,_Condiffpost, _AnchoredTrained, _AnchoredControl = \
    map(np.array, [_Tradiffpre,_Condiffpre,_Tradiffpost,_Condiffpost, _AnchoredTrained, _AnchoredControl])
    
    if len(Tradiffpre)==0:
        Tradiffpre = _Tradiffpre
        Condiffpre = _Condiffpre
        Tradiffpost = _Tradiffpost
        Condiffpost = _Condiffpost
        AnchoredTrainedDiff = _AnchoredTrained
        AnchoredControlDiff = _AnchoredControl
    else:
        Tradiffpre = np.vstack((Tradiffpre,_Tradiffpre))
        Condiffpre = np.vstack((Condiffpre,_Condiffpre))
        AnchoredTrainedDiff = np.vstack((AnchoredTrainedDiff,_AnchoredTrained))
        AnchoredControlDiff = np.vstack((AnchoredControlDiff,_AnchoredControl))
        Tradiffpost = np.vstack((Tradiffpost,_Tradiffpost))
        Condiffpost = np.vstack((Condiffpost,_Condiffpost))
        
UnanchoredTrainedDiff = Tradiffpost-Tradiffpre
UnanchoredControlDiff = Condiffpost-Condiffpre

Save out the differentiation measures as a DataFrame inside a csv.

In [9]:
labels = pd.DataFrame(data=sub_dirs, columns = ['IDs'])
MAT = np.hstack((AnchoredTrainedDiff.transpose(),AnchoredControlDiff.transpose(),
                 UnanchoredTrainedDiff.transpose(), UnanchoredControlDiff.transpose()))

CONDS = ['AnchoredTrainedDiff', 'AnchoredControlDiff',
         'UnanchoredTrainedDiff', 'UnanchoredControlDiff']
headers = []
for cond in CONDS:
    for roi in ROIs:
        headers.append('{}_{}'.format(cond,roi))
        
df = pd.DataFrame(data=MAT, columns = headers)
ids = pd.DataFrame(np.array(sub_dirs), columns = ['IDs'])
df = pd.concat([ids, df], axis = 1)
df.to_csv(os.path.join(results_dir, 'neural_changes_by_surfroi_and_subject.csv'))

## Sensitivity to measuring differences between object representations separately in each phase

Make sure that in each ROI and each phase the off-diagonal minus on-diagonal elements (btw-within) in each ROI > 0.  
This is a critical sanity check, as it indicates that the correlation distance between representations of *different* objects is larger than the correlation distance between the *same* object.  
This needs to be the case for looking at learning to make sense. 

In [12]:
sns.set_style('white')
sns.set_context('poster')

tmp = np.dstack((Tradiffpre,Condiffpre))
Diffpre = np.nanmean(tmp, axis=2)
tmp = np.dstack((Tradiffpost,Condiffpost))
Diffpost = np.nanmean(tmp, axis=2)

means_pre = np.nanmean(Diffpre, axis=1)
std_pre = np.nanstd(Diffpre, axis=1)/np.sqrt(shape(Diffpre)[1])

means_post = np.nanmean(Diffpost, axis=1)
std_post = np.nanstd(Diffpost, axis=1)/np.sqrt(shape(Diffpost)[1])

fig, ax = plt.subplots(figsize=(len(ROIs),6))

n_groups = len(ROIs) # num ROIs
index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.8
error_config = {'ecolor': '0.3'}

## plot means as bars
tcolor = (0.4,0.4,0.4)
rects1 = plt.bar(index, means_pre, bar_width,
                 alpha=opacity,
                 color=(0.4,0.4,0.4),
                 yerr=std_pre,
                 error_kw=error_config,
                 label='Pre')

ccolor = (0.7,0.7,0.7)
rects2 = plt.bar(index + bar_width, means_post, bar_width,
                 alpha=opacity,
                 color=(0.7,0.7,0.7),
                 yerr=std_post,
                 error_kw=error_config,
                 label='Post')


plt.xlabel('ROIs')
plt.ylabel('Btw vs. W/in-Obj Distance')
plt.title('Sensitivity to differences between object representations')
plt.xticks(index + bar_width / 2, ROIs_formatted)
plt.legend()
plt.tight_layout()
plt.savefig('./plots/sensitivity_to_measuring_distances.png')
plt.close()

## Learning-related changes in representational distance

Plot prepost representational differentiation for each ROI.  
One plot (4 total) for each possible configuration of these parameters drawn:
- Anchored vs Unanchored representational differentiation measure
- Separate results by condition vs plot the difference between conditions

### Unanchored Representational Distance

Plot prepost representational differentiation for each ROI, **separating results by condition**. 

In [17]:
## Generate summary plot (main analysis; old distance measure)

plot_indiv_subs = 0

sns.set_style('white')
sns.set_context('poster')

means_trained = np.nanmean(UnanchoredTrainedDiff, axis=1)
std_trained = np.nanstd(UnanchoredTrainedDiff, axis=1)/np.sqrt(shape(UnanchoredTrainedDiff)[1])

means_control = np.nanmean(UnanchoredControlDiff, axis=1)
std_control = np.nanstd(UnanchoredControlDiff, axis=1)/np.sqrt(shape(UnanchoredControlDiff)[1])

fig, ax = plt.subplots(figsize=(len(ROIs),6))

n_groups = len(ROIs) # num ROIs
index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.8
error_config = {'ecolor': '0.3'}

## plot means as bars
tcolor = (0.8,0.4,0.4)
rects1 = plt.bar(index, means_trained, bar_width,
                 alpha=opacity,
                 color=(0.8,0.4,0.4),
                 yerr=std_trained,
                 error_kw=error_config,
                 label='Trained')

ccolor = (0.4,0.4,0.4)
rects2 = plt.bar(index + bar_width, means_control, bar_width,
                 alpha=opacity,
                 color=(0.4,0.4,0.4),
                 yerr=std_control,
                 error_kw=error_config,
                 label='Control')

if plot_indiv_subs:
    ## now plot individual subjects as dots
    def generate_concat_tiled(array,reps):        
        inds = []
        for i in index:
            inds.append(np.tile(i,reps))
        return np.reshape(np.array(inds),(1,reps*len(array)))

    tindex = generate_concat_tiled(index,len(UnanchoredTrainedDiff[0]))
    tsubdists = np.reshape(UnanchoredTrainedDiff,(1,shape(UnanchoredTrainedDiff)[0]*shape(UnanchoredTrainedDiff)[1]))
    plt.scatter(tindex,tsubdists,s=25,alpha=0.2,color=tcolor)

    cindex = generate_concat_tiled(index,len(UnanchoredControlDiff[0]))+bar_width
    csubdists = np.reshape(UnanchoredControlDiff,(1,shape(UnanchoredControlDiff)[0]*shape(UnanchoredControlDiff)[1]))
    plt.scatter(cindex,csubdists,s=25,alpha=0.2,color=ccolor)

plt.xlabel('ROIs')
plt.ylabel('Change in Representational DISTANCE\n(YES if Trained reliably more positive than Control)')
plt.title('Effect of Training on Representational DISTANCE:\n"Are representational distances between object categories\nHIGHER after training compared to our control?"')
plt.xticks(index + bar_width / 2, ROIs_formatted)
plt.legend()
plt.tight_layout()
plt.savefig('./plots/unanchored_differentiation_by_roicondition.png')
plt.close()

Plot prepost representational differentiation for each ROI, ** now taking the difference between conditions**. 

In [18]:
## Generate summary plot (main analysis; old distance measure)
plot_indiv_subs = 0

sns.set_style('white')
sns.set_context('poster')
diffdiff = UnanchoredTrainedDiff-UnanchoredControlDiff

fig, ax = plt.subplots(figsize=(len(ROIs),6))

n_groups = len(ROIs) # num ROIs
index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.8
error_config = {'ecolor': '0.3'}

## plot means as bars
tcolor = (0.8,0.4,0.4)
rects1 = plt.bar(index, means_trained-means_control, bar_width,
                 alpha=opacity,
                 color=(0.8,0.4,0.4),
                 yerr=np.nanstd(diffdiff, axis=1)/np.sqrt(shape(diffdiff)[1]),
                 error_kw=error_config,
                 label='Trained')

ccolor = (0.4,0.4,0.4)

if plot_indiv_subs:
    ## now plot individual subjects as dots
    def generate_concat_tiled(array,reps):        
        inds = []
        for i in index:
            inds.append(np.tile(i,reps))
        return np.reshape(np.array(inds),(1,reps*len(array)))

    tindex = generate_concat_tiled(index,len(UnanchoredTrainedDiff[0]))
    tsubdists = np.reshape(UnanchoredTrainedDiff,(1,shape(UnanchoredTrainedDiff)[0]*shape(UnanchoredTrainedDiff)[1]))
    plt.scatter(tindex,tsubdists,s=25,alpha=0.2,color=tcolor)

    cindex = generate_concat_tiled(index,len(UnanchoredControlDiff[0]))+bar_width
    csubdists = np.reshape(UnanchoredControlDiff,(1,shape(UnanchoredControlDiff)[0]*shape(UnanchoredControlDiff)[1]))
    plt.scatter(cindex,csubdists,s=25,alpha=0.2,color=ccolor)

plt.xlabel('ROIs')
plt.title('(Unanchored) Effect of Training on Representational DISTANCE:\n"Are representational distances between object categories\nHIGHER after training compared to our control?"')
plt.ylabel('Difference Between Trained and Control Condition\nChange in Representational DISTANCE\n(YES if Positive)')
plt.xticks(index + bar_width / 2,  ROIs_formatted)
plt.savefig('./plots/unanchored_differentiation_by_roi.png')
plt.close()

### Anchored Representational Distance

Plot prepost representational differentiation for each ROI, **separating results by condition**. 

In [20]:
## Generate summary plot (main analysis; old distance measure)
plot_indiv_subs = 0

sns.set_style('white')
sns.set_context('poster')

means_trained = np.nanmean(AnchoredTrainedDiff, axis=1)
std_trained = np.nanstd(AnchoredTrainedDiff, axis=1)/np.sqrt(shape(AnchoredTrainedDiff)[1])

means_control = np.nanmean(AnchoredControlDiff, axis=1)
std_control = np.nanstd(AnchoredControlDiff, axis=1)/np.sqrt(shape(AnchoredControlDiff)[1])

fig, ax = plt.subplots(figsize=(len(ROIs),6))

n_groups = len(ROIs)
index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.8
error_config = {'ecolor': '0.3'}

## plot means as bars
tcolor = (0.8,0.4,0.4)
rects1 = plt.bar(index, means_trained, bar_width,
                 alpha=opacity,
                 color=(0.8,0.4,0.4),
                 yerr=std_trained,
                 error_kw=error_config,
                 label='Trained')

ccolor = (0.4,0.4,0.4)
rects2 = plt.bar(index + bar_width, means_control, bar_width,
                 alpha=opacity,
                 color=(0.4,0.4,0.4),
                 yerr=std_control,
                 error_kw=error_config,
                 label='Control')

if plot_indiv_subs:
    ## now plot individual subjects as dots
    def generate_concat_tiled(array,reps):        
        inds = []
        for i in index:
            inds.append(np.tile(i,reps))
        return np.reshape(np.array(inds),(1,reps*len(array)))

    tindex = generate_concat_tiled(index,len(Trained[0]))
    tsubdists = np.reshape(Trained,(1,shape(Trained)[0]*shape(Trained)[1]))
    plt.scatter(tindex,tsubdists,s=25,alpha=0.2,color=tcolor)

    cindex = generate_concat_tiled(index,len(Control[0]))+bar_width
    csubdists = np.reshape(Control,(1,shape(Control)[0]*shape(Control)[1]))
    plt.scatter(cindex,csubdists,s=25,alpha=0.2,color=ccolor)

plt.xlabel('ROIs')
plt.ylabel('Change in Representational DISTANCE\n(YES if Trained reliably more positive than Control)')
plt.title('Effect of Training on Representational DISTANCE:\n"Are representational distances between object categories\nHIGHER after training compared to our control?"')
plt.xticks(index + bar_width / 2, ROIs_formatted)
plt.legend()
plt.tight_layout()
plt.savefig('./plots/anchored_differentiation_by_roicondition.png')
plt.close()

Plot prepost representational differentiation for each ROI, ** now taking the difference between conditions**. 

In [22]:
## Generate summary plot (main analysis)
plot_indiv_subs = 0

sns.set_style('white')
sns.set_context('poster')

diffdiff = AnchoredTrainedDiff - AnchoredControlDiff

fig, ax = plt.subplots(figsize=(len(ROIs),6))

n_groups = len(ROIs)
index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.8
error_config = {'ecolor': '0.3'}

## plot means as bars
tcolor = (0.8,0.4,0.4)
rects1 = plt.bar(index, means_trained-means_control, bar_width,
                 alpha=opacity,
                 color=(0.8,0.4,0.4),
                 yerr=np.nanstd(diffdiff, axis=1)/np.sqrt(shape(diffdiff)[1]),
                 error_kw=error_config,
                 label='Trained')

ccolor = (0.4,0.4,0.4)

if plot_indiv_subs:
    ## now plot individual subjects as dots
    def generate_concat_tiled(array,reps):        
        inds = []
        for i in index:
            inds.append(np.tile(i,reps))
        return np.reshape(np.array(inds),(1,reps*len(array)))

    tindex = generate_concat_tiled(index,len(Trained[0]))
    tsubdists = np.reshape(Trained,(1,shape(Trained)[0]*shape(Trained)[1]))
    plt.scatter(tindex,tsubdists,s=25,alpha=0.2,color=tcolor)

    cindex = generate_concat_tiled(index,len(Control[0]))+bar_width
    csubdists = np.reshape(Control,(1,shape(Control)[0]*shape(Control)[1]))
    plt.scatter(cindex,csubdists,s=25,alpha=0.2,color=ccolor)

plt.xlabel('ROIs')
plt.title('(Anchored) Effect of Training on Representational DISTANCE:\n"Are representational distances between object categories\nHIGHER after training compared to our control?"')
plt.ylabel('Difference Between Trained and Control Condition\nChange in Representational DISTANCE\n(YES if Positive)')
plt.xticks(index + bar_width / 2,  ROIs_formatted)
plt.savefig('./plots/anchored_differentiation_by_roi.png')
plt.close()